In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from scipy import spatial
p_raw = '/project/nsaru/raw/PySlake/'
p_work = '/project/nsaru/work/PySlake/'

In [ ]:
def within(circle,point):
    if np.sqrt(np.dot(circle[0:2]-point,circle[0:2]-point))<circle[2]:
        return True
    else:
        return False

In [ ]:
def nearest_cell(circles,centroids):
    cells_xy = circles[0][:,0:2]
    aggrs_xy = np.array(centroids)
    d = spatial.distance_matrix(cells_xy,aggrs_xy)
    cells_idx = []
    for id in range(d.shape[1]):
        i = np.argmin(d[:,id])
        if within(circles[0][i,:],aggrs_xy[id,:]):
            cells_idx.append(np.array([i,id]))#cell id, aggregate id
        
    return cells_idx

In [ ]:
def cell_ids(circles,cell_labels):
    circles[0][:,0] = np.round(circles[0][:,0],decimals=-2)
    cdf = pd.DataFrame(circles[0][:,0:2], columns=['x', 'y'])
    cdf = cdf.sort_values(by=['x','y'])#counts down by column
    cdf['x'][0:6]=1200
    cdf['x'][6:12]=1400
    cdf['x'][12:18]=1600
    cdf['x'][18:24]=1800
    cdf['x'][24:30]=2000
    cdf['x'][30:36]=2200
    cdf = cdf.sort_values(by=['x','y'])#counts down by column
    cdf['y'][0:6]=np.array([500,700,900,1100,1300,1500])
    cdf['y'][6:12]=np.array([500,700,900,1100,1300,1500])
    cdf['y'][12:18]=np.array([500,700,900,1100,1300,1500])
    cdf['y'][18:24]=np.array([500,700,900,1100,1300,1500])
    cdf['y'][24:30]=np.array([500,700,900,1100,1300,1500])
    cdf['y'][30:36]=np.array([500,700,900,1100,1300,1500])
    cdf = cdf.sort_values(by=['x','y'])#counts down by column
    idx = np.array(cdf.index)
    labels = cell_labels.values[idx,:]
    return idx, cdf,labels
    

In [ ]:
all_samples = []
all_matches = []
all_areas = []
all_circles = []
all_labels = []
all_times = []
for d in os.listdir(p_raw):
    
    all_samples.append(d)
    t=[]
    labels_t = []
    matches_t = []
    areas_t = []
    circles_t = []
    for f in os.listdir(os.path.join(p_raw,d)):
        if '.csv' in f:
            if f!='ImageInfo.csv':
                cell_labels = pd.read_csv(os.path.join(p_raw,d,f))
    j = 0
    p_img = os.path.join(p_raw,d,'Images')
    if not os.path.isdir(p_img): 
        p_img = os.path.join(p_raw,d,'images')

    for f in os.listdir(p_img):
        t.append(os.path.getctime(os.path.join(p_img,f)))
        img = cv2.imread(os.path.join(p_img,f))
        b,g,r = cv2.split(img)       
        b = cv2.GaussianBlur(b,(3,3),1)   
        ret,bt = cv2.threshold(b,100,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

        h,s,v = cv2.split(cv2.cvtColor(img,cv2.COLOR_BGR2HSV))
        ret,thresh = cv2.threshold(v,100,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        bt = bt-thresh
        
        circles = cv2.HoughCircles(bt,cv2.HOUGH_GRADIENT,2,210,param1=400,param2=20,minRadius=105,maxRadius=125)

        try:
            if circles.shape[1]<36:
                circles = cv2.HoughCircles(bt,cv2.HOUGH_GRADIENT,2,200,param1=400,param2=10,minRadius=100,maxRadius=120)
            if circles.shape[1]>36:
                circles = cv2.HoughCircles(bt,cv2.HOUGH_GRADIENT,2,200,param1=400,param2=30,minRadius=100,maxRadius=120)
            
            circles = np.uint16(np.around(circles))
            for i in circles[0,:]:
                # draw the outer circle
                cv2.circle(img,(i[0],i[1]),i[2],(0,0,255),5)          
            idx,cdf,labels = cell_ids(circles,cell_labels)
    
            contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
            # draw contours on the original image
            cv2.drawContours(image=img, contours=contours, contourIdx=-1, color=(255, 0, 0), thickness=5, lineType=cv2.LINE_AA)
            
            centers = []
            A = []
            for c in contours:
                # compute the center of the contour
                M = cv2.moments(c)
                if M["m00"]>0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    centers.append(np.array([cX,cY]))
                    A.append(M["m00"])  
            cells_idx = nearest_cell(circles,centers)
             
            if not os.path.isdir(os.path.join(p_work,d)): os.mkdir(os.path.join(p_work,d))
            fout = os.path.join(p_work,d,f.split('.')[0]+'_labeled.jpg')
            cv2.imwrite(fout,img)

            matches_t.append(cells_idx)
            areas_t.append(A)
            circles_t.append(circles)
            labels_t.append(labels)
            #assign areas to cells using cells_idx
            j = j+1
        except Exception as e:
            print(e)
            print(d+'/'+f)
            plt.subplot(211)
            plt.imshow(bt)
            plt.subplot(212)
            plt.imshow(img)
            plt.show()

    all_circles.append(circles_t)
    all_matches.append(matches_t)
    all_areas.append(areas_t)
    all_labels.append(labels_t)
    all_times.append(t)

labels